In [1]:
import numpy as np
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
import re
import xlrd
import csv
import PyPDF2
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span
from string import punctuation

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk import tokenize
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize, pos_tag

from collections import Counter
import math

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bristi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/bristi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/bristi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
import spacy

def get_entities(sent):
    nlp = spacy.load("en_core_web_sm")

    # Document processing
    doc = nlp(sent)
    ents = list(doc.ents)  # Named entities
    noun_chunks = list(doc.noun_chunks)  # Noun phrases

    # Initialize variables for entity storage
    ent1 = ""
    ent2 = ""
    adjectives = []
    adverbs = []
    numbers = []

    # Use named entities and noun chunks to better identify relevant entity pairs
    if ents:
        ent1 = ents[0].text  # Choose the first named entity as Entity1
        if len(ents) > 1:
            ent2 = ents[1].text  # Choose the second named entity as Entity2

    # If no named entities, use noun chunks
    if not ent1 and noun_chunks:
        ent1 = noun_chunks[0].text
        if len(noun_chunks) > 1:
            ent2 = noun_chunks[1].text

    # Extract adjectives, adverbs, and numbers
    for tok in doc:
        if tok.pos_ == "ADJ":
            adjectives.append(tok.text)
        elif tok.pos_ == "ADV":
            adverbs.append(tok.text)
        elif tok.pos_ == 'NUM':
            numbers.append(tok.text)

    return {
        'Entity1': ent1,
        'Entity2': ent2,
        'Adjectives': adjectives,
        'Adverbs': adverbs,
        'Numbers': numbers
    }

# Test the function with a complex sentence
sentence = "Dubai is one of the seven Emirates that constitute the United Arab Emirates (UAE) in the Gulf region with a population of over 3 million people."
print(get_entities(sentence))


{'Entity1': 'Dubai', 'Entity2': 'one', 'Adjectives': [], 'Adverbs': [], 'Numbers': ['one', 'seven', '3', 'million']}


In [8]:
import spacy

def get_entities(sent):
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text with spaCy
    doc = nlp(sent)

    # Initialize containers
    entities = []
    adjectives = []
    adverbs = []
    numbers = []

    # Loop over named entities to categorize them
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC", "ORG"]:  # Geographic, Locations, Organizations
            entities.append((ent.text, ent.label_))
        elif ent.label_ == "NUM":  # Numbers
            numbers.append(ent.text)

    # Loop over each token in the document for adjectives and adverbs
    for tok in doc:
        if tok.pos_ == "ADJ" and tok.dep_.endswith("mod"):
            adjectives.append(tok.text)
        if tok.pos_ == "ADV":
            adverbs.append(tok.text)

    # Prepare to return the first two identified entities if they exist
    ent1 = entities[0][0] if len(entities) > 0 else ""
    ent2 = entities[1][0] if len(entities) > 1 else ""

    return {
        'Entity1': ent1,
        'Entity2': ent2,
        'Adjectives': adjectives,
        'Adverbs': adverbs,
        'Numbers': numbers
    }

# Example usage
sentence = "Dubai is one of the seven Emirates that constitute the United Arab Emirates (UAE) in the Gulf region with a population of over 3 million people."
print(get_entities(sentence))


{'Entity1': 'Dubai', 'Entity2': 'Emirates', 'Adjectives': [], 'Adverbs': [], 'Numbers': []}


In [10]:
import spacy

def get_entity_pairs(sent):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sent)

    # Initialize containers for entities and numbers
    places = []
    demographics = []

    # Extract entities and demographic numbers
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC", "ORG"]:
            places.append(ent.text)
        elif ent.label_ == "NUM" and 'population' in sent[sent.find(ent.text) - 50: sent.find(ent.text) + 50]:
            demographics.append(ent.text)

    # Initialize pairs
    pairs = []
    
    # Pair places if more than one is identified
    if len(places) > 1:
        for i in range(len(places) - 1):
            pairs.append((places[i], places[i+1]))

    # Pair first place with demographic information if available
    if places and demographics:
        pairs.append((places[0], demographics[0] + " people"))

    return pairs

# Example usage
sentence = "Dubai is one of the seven Emirates that constitute the United Arab Emirates (UAE) in the Gulf region with a population of over 3 million people."
print(get_entity_pairs(sentence))


[('Dubai', 'Emirates'), ('Emirates', 'the United Arab Emirates'), ('the United Arab Emirates', 'Gulf')]


In [12]:
import spacy

def get_entity_pairs(sent):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sent)

    # Initialize containers for entities
    places = []
    demographics = []

    # Extract entities
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC", "ORG"]:
            places.append(ent.text)
        elif ent.label_ == "NUM" and 'population' in sent[sent.find(ent.text) - 50: sent.find(ent.text) + 50]:
            demographics.append(ent.text + " people")

    # Initialize pairs
    pairs = []

    # Find and pair specific place with demographics
    for place in places:
        for demographic in demographics:
            if 'population' in sent[sent.find(place):sent.find(demographic)]:
                pairs.append((place, demographic))
                break  # Break after the first relevant demographic info to avoid duplication

    # Pair places sequentially for additional context
    if len(places) > 1:
        for i in range(len(places) - 1):
            pairs.append((places[i], places[i+1]))

    return pairs

# Example usage
sentence = "Dubai is one of the seven Emirates that constitute the United Arab Emirates (UAE) in the Gulf region with a population of over 3 million people."
print(get_entity_pairs(sentence))


[('Dubai', 'Emirates'), ('Emirates', 'the United Arab Emirates'), ('the United Arab Emirates', 'Gulf')]


In [13]:
import spacy

def get_entity_pairs(sent):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sent)

    # Initialize containers for entities
    places = {}
    demographics = []

    # Extract entities and demographic numbers
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC", "ORG"]:
            places[ent.text] = ent.start_char  # Store the place and its start position
        elif ent.label_ == "NUM" and 'population' in sent[sent.find(ent.text) - 50: sent.find(ent.text) + 50]:
            demographics.append((ent.text, ent.start_char))

    # Initialize pairs
    pairs = []
    
    # Find closest geographic entity for each demographic number
    for num, num_pos in demographics:
        closest_place = None
        min_distance = float('inf')
        for place, place_pos in places.items():
            distance = abs(place_pos - num_pos)
            if distance < min_distance:
                min_distance = distance
                closest_place = place
        if closest_place:
            pairs.append((closest_place, num + " people"))

    # Additionally, pair places sequentially if more than one is identified
    sorted_places = sorted(places.keys())
    if len(sorted_places) > 1:
        for i in range(len(sorted_places) - 1):
            pairs.append((sorted_places[i], sorted_places[i+1]))

    return pairs

# Example usage
sentence = "Dubai is one of the seven Emirates that constitute the United Arab Emirates (UAE) in the Gulf region with a population of over 3 million people."
print(get_entity_pairs(sentence))


[('Dubai', 'Emirates'), ('Emirates', 'Gulf'), ('Gulf', 'the United Arab Emirates')]
